# Harrisonburg, VA: Ideal Locations for New Businesses (Restaurant & Contractor)

### **Problem Statement and Background**

#### New businesses are always being created as new ideas for ways to serve the world are generated.  When setting up a new business, it can be challenging where to determine where to set up the physical office or establishment.  Customer quantity is of course one huge factor in the success of a business, so you want your office or business establishment to be close to where many potential customers will see and visit.  Customer stasification is another huge factor to success - people will keep coming back and bring friends if they have an enjoyable and memorable experience - and as a result, discounting the service of your business itself, you will want to be situated in a heart-warming, safe, and fun atmosphere.  Your service could be great, but it could be in a hard-to-reach or otherwise undesirable environment.  

#### This is where *location data* comes in.  We have two new business owners: one who is establishing a new restaurant, and one who is establishing a new building contractor firm office.  We will explore where the most ideal places are to set up a new restaurant and a new building contractor firm would be in Harrisonburg, Virginia, as of early-mid 2020.

### **Data**

##### This data analysis will consist of using data from **Foursquare API** and other sources, and will be broken up in two primary parts: the first for where a new restaurant could go, and the second for a new building contractor firm office.

#### For both new businesses, some esscentials will be knowing where the most common venues already exist.  More venues in a nearby area will attract more potential customers, but it may also mean more difficult competition if those venues are of the same business, so we will need to take that into consideration as well.  Likewise, we will want to consider other location factors for a new business, such as: foot traffic, accessibility and parking, image and history of the site, ordinances, and utility/rent/property taxes and such payments.

#### All of this data will be assembled from multiple sources and research, much of which will be placed in a CSV file, and will be used in tandem with Foursquare's API location venue data to determine the ideal location for both new businesses.

##### As a start, below is a brief look into the most common venues in Harrisonburg, VA.  All analysis is run using Python.

##### ------------------------------

##### *Import needed libraries.*

In [1]:
import pandas as pd
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np

##### *Import data table containing Harrisonburg's latitude and longitude*

In [3]:
# import data table from CSV file
import csv
with open('Harrisonburg_Coordinates.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
df = pd.read_csv('Harrisonburg_Coordinates.csv')
df.head()

,City,Latitude,Longitude
0,Harrisonburg,38.449569,-78.868916


##### *Connect to the Foursquare API*

In [4]:
CLIENT_ID = 'FTGWFBW5V4DDR3DGJHIUTOMA1U2Z2CPGBX32O4DYDWNSPOZX' # Foursquare ID
CLIENT_SECRET = 'T1YX0SIDMGB5GKCVORNDNYJH4URQRSCW0HPK1NI0CCAM35U3' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

##### *Create function that obtains venues in Harrisonburg*

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City',
                  'City Latitude',
                  'City Longitude',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
harrisonburg_venues = getNearbyVenues(names=df['City'], latitudes=df['Latitude'], longitudes=df['Longitude'])
harrisonburg_venues.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harrisonburg,38.449569,-78.868916,Jack Brown's Beer & Burger Joint,38.448605,-78.868644,Burger Joint
1,Harrisonburg,38.449569,-78.868916,Billy Jack's Wing and Draft Shack,38.448430,-78.868762,Bar
2,Harrisonburg,38.449569,-78.868916,Bella Gelato & Pastries,38.448489,-78.869982,Café
3,Harrisonburg,38.449569,-78.868916,Jimmy Madison's,38.448077,-78.869275,Southern / Soul Food Restaurant
4,Harrisonburg,38.449569,-78.868916,Bella Luna Wood Fired Pizza,38.448714,-78.870291,Pizza Place


In [10]:
print('There are {} unique categories of venue.'.format(len(harrisonburg_venues['Venue Category'].unique())))

There are 31 unique categories of venue.


##### *Frequency of occurrence of venues in Harrisonburg*

###### *(This code is also good for a dataset that contains more than one city.)*

In [11]:
# one hot encoding
harrisonburg_onehot = pd.get_dummies(harrisonburg_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
harrisonburg_onehot['City'] = harrisonburg_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [harrisonburg_onehot.columns[-1]] + list(harrisonburg_onehot.columns[:-1])
harrisonburg_onehot = harrisonburg_onehot[fixed_columns]

harrisonburg_grouped = harrisonburg_onehot.groupby('City').mean().reset_index()
harrisonburg_grouped

,City,American Restaurant,Arcade,Asian Restaurant,Bank,Bar,Beer Store,Bike Shop,Brewery,Burger Joint,...,Motel,Museum,New American Restaurant,Pizza Place,Plaza,Seafood Restaurant,Snack Place,Southern / Soul Food Restaurant,Theater,Wine Shop
0,Harrisonburg,0.04878,0.02439,0.02439,0.02439,0.121951,0.02439,0.02439,0.073171,0.02439,...,0.02439,0.02439,0.04878,0.04878,0.02439,0.02439,0.02439,0.02439,0.02439,0.02439


##### *Retrieve venue categories in Harrisonburg*

In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### *View the most common venues in Harrisonburg*

In [14]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
harrisonburg_venues_sorted = pd.DataFrame(columns=columns)
harrisonburg_venues_sorted['City'] = harrisonburg_grouped['City']

for ind in np.arange(harrisonburg_grouped.shape[0]):
    harrisonburg_venues_sorted.iloc[ind, 1:] = return_most_common_venues(harrisonburg_grouped.iloc[ind, :], num_top_venues)

harrisonburg_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Harrisonburg,Bar,Brewery,American Restaurant


### We see that the three most common venues in Harrisonburg are Bars, Brewerys, and American Restaurants, which means we'll have to be extra careful about where we place that new restaurant of ours.  One thing's for sure - this city loves to drink.

##### This concludes the introduction to this analysis.  The next and final part will contain the analysis for both new businesses.